# Lab1


In [ ]:
# Read the Input File
input_file = open("Input file.txt", "r")

# Declare 2 Dictionary
heuristic = {}
neighbors = {}


for i in input_file:
  temp = i.split(" ")

  # Store The Heuristic Values
  heuristic[temp[0]] = int(temp[1])

  # Store The Neighbor's Information
  neighbors[temp[0]] = []
  for j in range(2,len(temp)-1,2):
    city = temp[j]
    distance = temp[j+1]
    neighbors[temp[0]].append((city,int(distance)))

input_file.close()

# print(heuristic)
# print(neighbors)

# Taking Input from the user
start = input("Start node: ")
goal = input("Destination: ")

# Find the optimum path

import heapq

def printpath(parent, start, goal, distance):
  # If goal was never found
  if goal not in parent:
    print("NO PATH FOUND")
    return

  # If goal was found
  path = []
  temp = goal

  while temp is not None:
    path.append(temp)
    temp = parent[temp]

  path.reverse()
  print(f"Path: {' -> '.join(path)}")
  print(f"Total distance: {distance[goal]} km")


# A Star Algorithm

def ASterSearch(start, goal):
  if start not in heuristic or goal not in heuristic:
    print("NO PATH FOUND")
    return

  parent = {start: None}  # Track the optimal path
  distance = {start: 0}   # Calculate the actual distance
  queue = []  # priority queue
  heapq.heappush(queue, (heuristic[start], start))

  while queue:
    temp = heapq.heappop(queue)    #pop the city of lowest f(n) value

    # If we reach the goal, print the path
    if temp[1] == goal:
      printpath(parent, start, goal, distance)
      return

    # Using .get() function to avoid KeyError
    for i in neighbors.get(temp[1], []):
      temp_d = distance[temp[1]]+ i[1]         #Actual path cost from start

      if i[0] not in distance or temp_d < distance[i[0]]:
        distance[i[0]] = temp_d

        # Handle missing heuristic values to avaoid KeyError by default infinity value

        # Formula = f(n) = g(n)+h(n)
        astar_cost = distance[i[0]] + heuristic.get(i[0], float('inf'))   # distance(i) = g(start) + total distance(start -> i) + h(i)

        heapq.heappush(queue, (astar_cost, i[0]))
        parent[i[0]] = temp[1]

  # If we finish the loop without reaching the goal
  print("NO PATH FOUND")

ASterSearch(start, goal)


Start node: Arad
Destination: Bucharest
Path: Arad -> Sibiu -> RimnicuVilcea -> Pitesti -> Bucharest
Total distance: 418 km


# Lab2


In [ ]:
import random

# Part-1: Genetic Algorithm with Single Point Crossover & Mutation

# Function to convert integers to 2-digit gene strings and form a chromosome
def chromosome_structure(list1, x, y, z):
    if x < 10:
        Sl = "0" + str(x)
    else:
        Sl = str(x)
    if y < 10:
        Tp = "0" + str(y)
    else:
        Tp = str(y)
    if z < 10:
        Ts = "0" + str(z)
    else:
        Ts = str(z)

    new_chromosome = Sl + Tp + Ts  # Combine genes into 6-digit string
    list1.append(new_chromosome)  # Add to population list
    return

# Function to calculate fitness value for each chromosome
def Fitness_Value(list2, dict, historical_prices, Capital_to_Start):
    start_price = Capital_to_Start
    for i in range(len(list2)):
        Sl = (list1[i][0:2])
        Sl = -(int(Sl))  # Stop-loss as negative threshold
        Tp = (list1[i][2:4])
        Tp = int(Tp)  # Take-profit
        Ts = (list1[i][4:6])
        Ts = int(Ts)  # Trade size %

        for price in historical_prices:
            Trade_Size = (start_price * (Ts / 100))
            if price < Sl:
                profit = Trade_Size * (Sl / 100)  # stop-loss triggered
            elif price > Tp:
                profit = Trade_Size * (Tp / 100)  # take-profit triggered
            else:
                profit = Trade_Size * (price / 100)  # normal change
            start_price += profit

        T_profit = start_price - Capital_to_Start  # Net gain
        dict[list2[i]] = T_profit  # Save in dictionary
    return

# Function to apply mutation to each chromosome randomly
def mutation(child_list):
    for i in range(len(child_list)):
        mutant = child_list[i]
        value = random.randint(1, 99)  # New random value
        point = random.choice([0, 2, 4])  # Randomly choose gene to mutate

        if point == 0:  # Mutate stop-loss
            if value < 10:
                new = "0" + str(value) + mutant[2:]
            else:
                new = str(value) + mutant[2:]
        elif point == 2:  # Mutate take-profit
            if value < 10:
                new = mutant[0:2] + "0" + str(value) + mutant[4:]
            else:
                new = mutant[0:2] + str(value) + mutant[4:]
        else:  # Mutate trade-size
            if value < 10:
                new = mutant[0:4] + "0" + str(value)
            else:
                new = mutant[0:4] + str(value)

        child_list[i] = new  # Replace mutated chromosome
    return child_list

# Function to perform single-point crossover
def crossover(p1, p2):
    point = random.randint(1, 5)  # Random crossover point
    new1 = p1[0:point] + p2[point:]
    new2 = p2[0:point] + p1[point:]
    return new1, new2

# Function to select random parents, apply crossover and mutation
def select_parents(list1):
    list2 = []
    for i in range(2):
        new1, new2 = crossover(list1[random.randint(0, 3)], list1[random.randint(0, 3)])
        list2.append(new1)
        list2.append(new2)
    childs = mutation(list2)  # Apply mutation
    return childs

# Initialize population
list1 = []
dict = {}      #store {chromosome:total profit}
for i in range(4):  # Generate 4 chromosomes
    Sl = random.randint(1, 99)
    Tp = random.randint(1, 99)
    Ts = random.randint(1, 99)
    chromosome_structure(list1, Sl, Tp, Ts)

# Historical data and capital
historical_prices = [-1.2, 3.4, -0.8, 2.1, -2.5, 1.7, -0.3, 5.8, -1.1, 3.5]
Capital_to_Start = 1000
generations = 10

# Initial fitness calculation
Fitness_Value(list1, dict, historical_prices, Capital_to_Start)

# Genetic Algorithm loop
for i in range(generations):
    new_population = select_parents(list1)
    Fitness_Value(new_population, dict, historical_prices, Capital_to_Start)
    list1 = new_population  # Update population

# Select best performing chromosome
max_key = max(dict, key=dict.get)
max_value = dict[max_key]
stop_loss = (max_key[:2])
take_profit = (max_key[2:4])
trade_size = (max_key[4:])

# Output best strategy
output_file = open("output1.txt", "w")
output_file.write(f'Best Strategy:\n')
output_file.write(f'Stop Loss: {stop_loss}\n')
output_file.write(f'Take Profit: {take_profit}\n')
output_file.write(f'Trade Size: {trade_size}\n')
output_file.write(f'Final Profit: {max_value}\n')


# Part-2: Two-Point Crossover Function

def two_point_crossover(parent1, parent2):
    length = len(parent1)
    point1, point2 = sorted(random.sample(range(1, length), 2))  # Sorted to make sure point1 < point2

    offspring1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    offspring2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return offspring1, offspring2, point1, point2


# Demo test case for two-point crossover
parent1 = "000111000"
parent2 = "111000111"

offspring1, offspring2, point1, point2 = two_point_crossover(parent1, parent2)
output_file = open("output2.txt", "w")
output_file.write(f'Parent 1: {parent1}\n')
output_file.write(f'Parent 2: {parent2}\n')
output_file.write(f'Two-point crossover at indices: {point1}, {point2}\n')
output_file.write(f'Offspring 1: {offspring1}\n')
output_file.write(f'Offspring 2: {offspring2}\n')
output_file.close()

#Lab3

In [ ]:
#part1 - Alpha Beta Pruning with Minimax and Utility
import math
import random

# Calculates the player's strength using log scale + linear component
def strength(value):
    return (math.log2(value+1)) + (value / 10)

# Calculates the utility value with randomness for realistic game simulation
def utility_function(maxv,minv):
    i = random.choice([1,0])  # Randomly pick whether to add or subtract noise
    one = (-1)**i  # Either -1 or 1
    one = one * (random.randint(1,10)/10)  # Value between -1.0 to +1.0
    utility = strength(maxv) - strength(minv) + one  # Final utility
    return utility

# Minimax algorithm with alpha-beta pruning
def Minimax(depth, alpha, beta, player, maxV, minV):
    if depth == 0:
        return utility_function(maxV, minV)  # Base case: return utility

    if player == 0:  # Maximizer (MAX)
        result = float('-inf')
        for i in range(2):  # Binary tree, branching factor = 2
            new = Minimax(depth-1, alpha, beta, 1, maxV, minV)
            result = max(result, new)
            alpha = max(alpha, result)
            if alpha >= beta:  # Prune
                break
        return result

    else:  # Minimizer (MIN)
        result = float('inf')
        for i in range(2):
            new = Minimax(depth-1, alpha, beta, 0, maxV, minV)
            result = min(result, new)
            beta = min(beta, result)
            if alpha >= beta:  # Prune
                break
        return result

# Starts the game and runs 4 matches, alternating players
def start_game(start, player1_strenght, player2_strenght):
    depth = 5
    Total_games = 4
    result = {"Carlsen": 0, "Caruana": 0, "Draw": 0}  # Scoreboard
    details = []
    alpha = float('-inf')
    beta = float('inf')

    for i in range(Total_games):
        if start == 0:
            maxV = player1_strenght
            minV = player2_strenght
            max_player = "Carlsen"
            min_player = "Caruana"
        else:
            maxV = player2_strenght
            minV = player1_strenght
            max_player = "Caruana"
            min_player = "Carlsen"

        utility = Minimax(depth, alpha, beta, start, maxV, minV)
        print(utility, max_player, min_player)

        if utility < 0:
            winner = min_player
            winner_type = "Min"
            result[winner] += 1
        elif utility > 0:
            winner = max_player
            result[winner] += 1
            winner_type = "Max"
        else:
            winner = None
            winner_type = "Draw"

        details.append((i+1, winner, winner_type, utility))
        start = 1 - start  # Alternate who plays first

    return result, details

# Inputs for the 4-game simulation
start = int(input("Enter starting player for game 1 (0 for Carlsen, 1 for Caruana): "))
B_carlsen = int(input("Enter base strength for Carlsen: "))
B_caruana = int(input("Enter base strength for Caruana: "))

# Run game
result, details = start_game(start, B_carlsen, B_caruana)

# Determine overall winner
if result["Carlsen"] > result["Caruana"]:
    winner = "Carlsen"
else:
    winner = "Caruana"

# Print game-wise and overall results
for i in details:
    game, winny, type, utility = i
    print(f"Game {game} Winner: {winny} ({type}) (Utility value: {utility:.2f})")

print("Overall Results: ")
print("Magnus Carlsen Wins:", result["Carlsen"])
print("Fabiano Caruana Wins:", result["Caruana"])
print("Draws:", result["Draw"])
print("overall winner:", winner)


#part2 - Mind Control Simulation

# Modified minimax where even Min player is controlled by Max (i.e., both maximize)
def mind_control(depth, is_max, alpha, beta, maxV, minV):
    if depth == 0:
        return utility_function(maxV, minV)

    result = float('-inf')
    for i in range(2):
        new = mind_control(depth-1, alpha, beta, not is_max, maxV, minV)
        result = max(result, new)  # Both Max and Min maximize
        if is_max:
            alpha = max(alpha, result)
        else:
            beta = max(beta, result)
        if alpha >= beta:
            break
    return result

# Run normal and mind control minimax for 1 match and compare
def start_game(start, player1_strenght, player2_strenght):
    depth = 5
    alpha = float('-inf')
    beta = float('inf')

    if start == 0:
        maxV = player1_strenght
        minV = player2_strenght
        max_player = "Light"
        min_player = "L"
    else:
        maxV = player2_strenght
        minV = player1_strenght
        max_player = "L"
        min_player = "Light"

    utility = Minimax(depth, alpha, beta, start, maxV, minV)  # normal minimax
    mind_val = mind_control(depth, True, alpha, beta, maxV, minV)  # controlled version

    return utility, mind_val

# Inputs
start = int(input("Enter starting player for game 1 (0 for Light, 1 for L): "))
cost = float(input("Enter the cost of using Mind Control: "))
B_Light = int(input("Enter base strength for Light: "))
B_L = int(input("Enter base strength for L: "))

# Run comparison
result, mind_val = start_game(start, B_Light, B_L)

# Print logic and final recommendation
if start == 0:
    print("Minimax value without Mind Control:", result)
    print("Minimax value with Mind Control:", mind_val)
    print("Minimax value with Mind Control after incurring the cost:", (mind_val-cost))
    if result > 0:
        print("Light should NOT use Mind Control as the position is already winning")
    elif result < 0 and (mind_val-cost) > 0:
        print("Light should use Mind Control.")
    elif result < 0 and (mind_val-cost) < 0:
        print("Light should NOT use Mind Control as the position is losing either way.")
    else:
        print("Light should NOT use Mind Control as it backfires.")
else:
    print("Minimax value without Mind Control:", result)
    print("Minimax value with Mind Control:", mind_val)
    print("Minimax value with Mind Control after incurring the cost:", (mind_val-cost))
    if result > 0:
        print("L should NOT use Mind Control as the position is already winning")
    elif result < 0 and (mind_val-cost) > 0:
        print("L should use Mind Control.")
    elif result < 0 and (mind_val-cost) < 0:
        print("L should NOT use Mind Control as the position is losing either way.")
    else:
        print("L should NOT use Mind Control as it backfires.")


Enter starting player for game 1 (0 for Carlsen, 1 for Caruana): 0
Enter base strength for Carlsen: 9
Enter base strength for Caruana: 8
0.7520030934450501 Carlsen Caruana
-0.7520030934450501 Caruana Carlsen
0.8520030934450501 Carlsen Caruana
-0.8520030934450501 Caruana Carlsen
Game 1 Winner: Carlsen (Max) (Utility value: 0.75)
Game 2 Winner: Carlsen (Min) (Utility value: -0.75)
Game 3 Winner: Carlsen (Max) (Utility value: 0.85)
Game 4 Winner: Carlsen (Min) (Utility value: -0.85)
Overall Results: 
Magnus Carlsen Wins: 4
Fabiano Caruana Wins: 0
Draws: 0
overall winner: Carlsen
Enter starting player for game 1 (0 for Light, 1 for L): 0
Enter the cost of using Mind Control: 0.25
Enter base strength for Light: 10
Enter base strength for L: 8
Minimax value without Mind Control: 0.8895066171949854
Minimax value with Mind Control: 1.3895066171949852
Minimax value with Mind Control after incurring the cost: 1.1395066171949852
Light should NOT use Mind Control as the position is already winning